# Predicting YCombinator Startup Performance

### Load dataset

In [15]:
import pandas as pd
import os
from fuzzywuzzy import process
from fuzzywuzzy import fuzz
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

ModuleNotFoundError: No module named 'seaborn'

In [30]:
badges = pd.read_csv('archive/badges.csv')
companies = pd.read_csv('archive/companies.csv')
founders = pd.read_csv('archive/founders.csv')
industries = pd.read_csv('archive/industries.csv')
prior_companies = pd.read_csv('archive/prior_companies.csv')
regions = pd.read_csv('archive/regions.csv')
schools = pd.read_csv('archive/schools.csv')
tags = pd.read_csv('archive/tags.csv')

badges['badge_num'] = badges.groupby('id').cumcount() + 1
df_pivot = badges.pivot(index='id', columns='badge_num', values='badge')
df_pivot.columns = [f'badge_{int(col)}' for col in df_pivot.columns]
badges = df_pivot.reset_index()

industries['industry_num'] = industries.groupby('id').cumcount() + 1
df_pivot = industries.pivot(index='id', columns='industry_num', values='industry')
df_pivot.columns = [f'industry_{int(col)}' for col in df_pivot.columns]
industries = df_pivot.reset_index()

tags['tag_num'] = tags.groupby('id').cumcount() + 1
df_pivot = tags.pivot(index='id', columns='tag_num', values='tag')
df_pivot.columns = [f'tag_{int(col)}' for col in df_pivot.columns]
tags = df_pivot.reset_index()

schools = schools.drop_duplicates(subset='hnid', keep='first')
ivy_league_schools = [
    'Harvard University',
    'Yale University',
    'Princeton University',
    'Columbia University',
    'University of Pennsylvania',
    'Dartmouth College',
    'Brown University',
    'Cornell University'
]

#TODO why all false?
def is_ivy_league(school_name):
    # Strip whitespace and use fuzzy matching
    school_name = str(school_name)
    school_name = school_name.strip().lower()
    # Get the best match and its score
    best_match, score = process.extractOne(school_name, ivy_league_schools, scorer=fuzz.ratio)
    return score >= 80

schools['is_ivy_league'] = schools['school'].apply(is_ivy_league)
faang_companies = [
    'Facebook',
    'Apple',
    'Meta',
    'Amazon',
    'Netflix',
    'Microsoft',
    'Google'
]
def is_faang(company_name):
    company_name = str(company_name)
    company_name = company_name.strip().lower()
    best_match, score = process.extractOne(company_name, faang_companies, scorer=fuzz.ratio)
    return score >= 80  # Threshold for matching

prior_companies['is_faang'] = prior_companies['company'].apply(is_faang)

founder_dfs = [founders, schools, prior_companies]
company_dfs = [badges, companies, industries, regions, tags]

for i, df in enumerate(company_dfs):
    if 'Unnamed: 0' in df.columns:
        df.drop(columns=['Unnamed: 0'], inplace=True)

merged_df = company_dfs[0]

for i, df in enumerate(company_dfs[1:], start=1):
    merged_df = pd.merge(merged_df, df, on='id', how='inner')

merged_df.rename(columns={'slug': 'company_slug'}, inplace=True)
#merged_df = pd.merge(merged_df, founders, on='company_slug', how='inner')
#merged_df = pd.merge(merged_df, schools, on='hnid', how='inner')
#merged_df = pd.merge(merged_df, prior_companies, on='hnid', how='inner')

merged_df


,id,badge_1,badge_2,badge_3,badge_4,name,company_slug,website,smallLogoUrl,oneLiner,...,industry_2,region,country,address,tag_1,tag_2,tag_3,tag_4,tag_5,tag_6
0,5,isHiring,NaN,NaN,NaN,CircuitHub,circuithub,https://circuithub.com,https://bookface-images.s3.amazonaws.com/small...,Electronics factory automation,...,Manufacturing and Robotics,Europe,United Kingdom,"London, UK",Hard Tech,Hardware,Robotics,NaN,NaN,NaN
1,8,topCompany,highlightWomen,NaN,NaN,PlanGrid,plangrid,http://plangrid.com,https://bookface-images.s3.amazonaws.com/small...,Mobile applications for the construction indus...,...,Construction,America / Canada,United States of America,"San Francisco, CA, USA",Construction,NaN,NaN,NaN,NaN,NaN
2,10,highlightWomen,NaN,NaN,NaN,The Muse,the-muse,http://themuse.com,https://bookface-images.s3.amazonaws.com/small...,The Muse is values-based job search & hiring f...,...,Recruiting and Talent,America / Canada,United States of America,"New York, NY, USA",Marketplace,SaaS,Recruiting,NaN,NaN,NaN
3,11,highlightBlack,NaN,NaN,NaN,SendHub,sendhub,http://sendhub.com,https://bookface-images.s3.amazonaws.com/small...,Simple Business SMS. SMBs to Large Enterprise.,...,Productivity,America / Canada,United States of America,"Pleasanton, CA, USA",Messaging,Enterprise,SMB,NaN,NaN,NaN
4,19,isHiring,NaN,NaN,NaN,Rescale,rescale,https://rescale.com,https://bookface-images.s3.amazonaws.com/small...,High Performance Computing Built for the Cloud,...,"Engineering, Product and Design",America / Canada,United States of America,"San Francisco, CA, USA",Cloud Computing,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1978,29964,isHiring,NaN,NaN,NaN,Overlap,overlap,https://www.joinoverlap.com/,https://bookface-images.s3.amazonaws.com/small...,Overlap is Perplexity for video & audio content,...,Content,America / Canada,United States of America,"San Francisco, CA, USA",Consumer,Media,Podcasts,AI,NaN,NaN
1979,29967,highlightBlack,NaN,NaN,NaN,Village Labs,village-labs,https://villagelabs.app/,https://bookface-images.s3.amazonaws.com/small...,Helping America's businesses become employee o...,...,Human Resources,America / Canada,United States of America,"New York, NY, USA",Fintech,Human Resources,NaN,NaN,NaN,NaN
1980,29972,highlightBlack,NaN,NaN,NaN,Affil.ai,affil-ai,https://affil.ai/,https://bookface-images.s3.amazonaws.com/small...,AI Affiliate Network For Financial Companies,...,Marketing,America / Canada,United States of America,"San Francisco, CA, USA",SaaS,B2B,Compliance,Marketing,AI,NaN
1981,29977,highlightWomen,NaN,NaN,NaN,Merlin AI,merlin-ai,https://www.merlinai.co,https://bookface-images.s3.amazonaws.com/small...,AI powered ERP for Construction,...,NaN,America / Canada,United States of America,"Los Angeles, CA, USA",SaaS,Construction,B2B,Enterprise Software,AI,NaN


### Removing the duplicates

In [29]:
def choose_row(group):
    na_counts = group.isna().sum(axis=1)
    return group.loc[na_counts.idxmin()]

merged_df = merged_df.groupby('name').apply(choose_row).reset_index(drop=True)
merged_df

/var/folders/5m/cnfn53b56j56_vqxvwq4djgc0000gn/T/ipykernel_60948/2443777121.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  merged_df = merged_df.groupby('name').apply(choose_row).reset_index(drop=True)


,id,badge_1,badge_2,badge_3,badge_4,name,company_slug,website,smallLogoUrl,oneLiner,...,industry_2,region,country,address,tag_1,tag_2,tag_3,tag_4,tag_5,tag_6
0,29798,highlightBlack,NaN,NaN,NaN,&AI,ai-2,https://www.tryandai.com/,https://bookface-images.s3.amazonaws.com/small...,The secure patent due diligence and management...,...,Legal,America / Canada,United States of America,"San Francisco, CA, USA",SaaS,B2B,LegalTech,NaN,NaN,NaN
1,1700,highlightWomen,NaN,NaN,NaN,10 By 10,10-by-10,https://10by10.io,https://bookface-images.s3.amazonaws.com/small...,10by10 is a unique marketplace in the recruiti...,...,Recruiting and Talent,America / Canada,United States of America,"San Francisco, CA, USA",Artificial Intelligence,Marketplace,Recruiting,NaN,NaN,NaN
2,12286,highlightBlack,highlightWomen,NaN,NaN,1910 Genetics,1910-genetics,http://1910genetics.com,https://bookface-images.s3.amazonaws.com/small...,Make undruggable targets a thing of the past,...,Therapeutics,America / Canada,United States of America,"Cambridge, MA, USA",AI-powered Drug Discovery,NaN,NaN,NaN,NaN,NaN
3,27925,isHiring,NaN,NaN,NaN,222,222,https://222.place,https://bookface-images.s3.amazonaws.com/small...,marketplace facilitating IRL social experience...,...,NaN,America / Canada,United States of America,"New York, NY, USA",Machine Learning,Marketplace,Consumer,Social,AI,NaN
4,12252,highlightBlack,NaN,NaN,NaN,54Gene,54gene,http://www.54gene.com,https://bookface-images.s3.amazonaws.com/small...,Precision medicine for Africans and the global...,...,Consumer Health and Wellness,Africa,Nigeria,"Lagos, Nigeria",Genomics,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1960,25691,isHiring,NaN,NaN,NaN,voize,voize,https://voize.de/,https://bookface-images.s3.amazonaws.com/small...,App for healthcare workers to fill out forms u...,...,Productivity,Europe,Germany,"Berlin, Germany",Healthcare,NaN,NaN,NaN,NaN,NaN
1961,21906,highlightWomen,NaN,NaN,NaN,webapp.io,webapp-io,https://webapp.io,https://bookface-images.s3.amazonaws.com/small...,Build webapps faster with 10x better CI/CD + p...,...,"Engineering, Product and Design",America / Canada,Canada,"Toronto, ON, Canada",Developer Tools,SaaS,B2B,NaN,NaN,NaN
1962,22856,isHiring,NaN,NaN,NaN,weweb.io,weweb-io,https://www.weweb.io/,https://bookface-images.s3.amazonaws.com/small...,WeWeb is a no-code front-end builder that conn...,...,Operations,Europe,France,"Paris, France",Developer Tools,Web Development,NaN,NaN,NaN,NaN
1963,25977,highlightWomen,NaN,NaN,NaN,yhangry,yhangry,http://www.yhangry.com,https://bookface-images.s3.amazonaws.com/small...,Marketplace for private chefs,...,Food and Beverage,Europe,United Kingdom,"London, UK",Marketplace,NaN,NaN,NaN,NaN,NaN


### Create variables

In [34]:
def choose_row(group):
    na_counts = group.isna().sum(axis=1)
    return group.loc[na_counts.idxmin()]

merged_df = merged_df.groupby('name').apply(choose_row).reset_index(drop=True)

print("DataFrame after removing duplicates with the most NaNs:")
print(merged_df)

DataFrame after removing duplicates with the most NaNs:
         id          badge_1         badge_2 badge_3 badge_4           name  \
0     29798   highlightBlack             NaN     NaN     NaN            &AI   
1      1700   highlightWomen             NaN     NaN     NaN       10 By 10   
2     12286   highlightBlack  highlightWomen     NaN     NaN  1910 Genetics   
3     27925         isHiring             NaN     NaN     NaN            222   
4     12252   highlightBlack             NaN     NaN     NaN         54Gene   
...     ...              ...             ...     ...     ...            ...   
1960  25691         isHiring             NaN     NaN     NaN          voize   
1961  21906   highlightWomen             NaN     NaN     NaN      webapp.io   
1962  22856         isHiring             NaN     NaN     NaN       weweb.io   
1963  25977   highlightWomen             NaN     NaN     NaN        yhangry   
1964  29532  highlightLatinx             NaN     NaN     NaN    Ångström AI

/var/folders/5m/cnfn53b56j56_vqxvwq4djgc0000gn/T/ipykernel_60948/1798533604.py:9: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  merged_df = merged_df.groupby('name').apply(choose_row).reset_index(drop=True)
